In [30]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

from pypfopt import EfficientFrontier, risk_models, objective_functions, BlackLittermanModel, black_litterman

In [4]:
historical_returns = pd.read_csv('./sample/blm_asset_returns.csv', parse_dates=True, index_col = 'Year')
weights = pd.read_csv('./sample/blm_asset_weights.csv', parse_dates=True, index_col = 'asset_class')

In [5]:
# Select some asset classes
cols = [
    'Global Bonds (Unhedged)', 
    'Total US Bond Market', 
    'US Large Cap Growth', 
    'US Large Cap Value', 
    'US Small Cap Growth', 
    'US Small Cap Value', 
    'Emerging Markets', 
    'Intl Developed ex-US Market', 
    'Short Term Treasury'
]

returns = historical_returns[cols].dropna()
treasury_rate = returns['Short Term Treasury']

# exclude Treasury Rate from calculation
returns = returns.drop('Short Term Treasury', axis = 1).astype(np.float).dropna()
weights = weights.loc[cols[:-1]]

/Users/pasin/opt/anaconda3/envs/env01/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [6]:
# Avg returns
returns.mean()

Global Bonds (Unhedged)        0.059615
Total US Bond Market           0.054231
US Large Cap Growth            0.131538
US Large Cap Value             0.102308
US Small Cap Growth            0.119615
US Small Cap Value             0.114615
Emerging Markets               0.100000
Intl Developed ex-US Market    0.063462
dtype: float64

In [7]:
# Excess Return
excess_returns = returns.subtract(treasury_rate, axis = 0)

# Covariance Matrix
cov = excess_returns.cov() 

# Global Return (Return of asset class * weights)
global_return = excess_returns.mean().multiply(weights['weight'].values).sum()

# Market Variance
weight_vector = weights.values.reshape(len(weights))
market_var = weight_vector.T @ cov @ weight_vector

print(f'Global Market Average Return is {global_return:.4f} with Market Variance equal to {market_var:.6}')

risk_aversion_coef = global_return / market_var
print(f'The Risk Aversion Coefficient is {risk_aversion_coef:.2f}')

Global Market Average Return is 0.0446 with Market Variance equal to 0.0202548
The Risk Aversion Coefficient is 2.20


In [16]:
# Calculate implied returns 
def get_implied_returns(risk_aversion_coef, cov, weights):
    return risk_aversion_coef * (cov @ weights).squeeze()

implied_returns_eq = get_implied_returns(risk_aversion_coef=risk_aversion_coef, cov = cov, weights=weights)
implied_returns_eq

Global Bonds (Unhedged)        0.012871
Total US Bond Market           0.002439
US Large Cap Growth            0.060436
US Large Cap Value             0.051548
US Small Cap Growth            0.056798
US Small Cap Value             0.043902
Emerging Markets               0.076184
Intl Developed ex-US Market    0.063076
Name: weight, dtype: float64

In [17]:
# VIEWS
Q = np.array([0.0925, 0.005, 0.055])
P = np.array([
    [0, 0, 0, 0, 0, 0, 1, 0], 
    [0, 0, 0.85, -0.85, -0.15, -0.15, 0, 0], 
    [0, 0, 0, 0, 0, 0, 0, 1]
])

# Calculate each view variance
n_views = len(Q)
view_var = []
for i in range(n_views):
    view_vector = P[i].reshape(len(P[i]))
    view_cov = view_vector.T @ cov @ view_vector
    view_var.append(view_cov)
    print(f'Variance of view {i} is {view_cov:.3f}')


Variance of view 0 is 0.097
Variance of view 1 is 0.015
Variance of view 2 is 0.045


In [18]:
view_var

[0.09655215384615386, 0.01534823115384617, 0.04505784615384616]

In [19]:
# Find Omega
def get_error_cov_mat(cov, tau, P):
    return np.diag(
        np.diag(
            P @ (tau * cov) @ P.T
        )
    )

tau = 0.025
omega = get_error_cov_mat(cov = cov, tau = tau, P = P)

In [20]:
# Calculate View-based Return Vector
cov_scaled = cov * tau 
BLM_return_vector = implied_returns_eq + \
    cov_scaled @ P.T @ (
        np.linalg.inv(
            (P @ cov_scaled @ P.T) + omega 
        ) @ (Q - (
            P @ implied_returns_eq
        ))
    )
BLM_return_vector

Global Bonds (Unhedged)        0.013137
Total US Bond Market           0.002297
US Large Cap Growth            0.061663
US Large Cap Value             0.046864
US Small Cap Growth            0.054727
US Small Cap Value             0.037481
Emerging Markets               0.079109
Intl Developed ex-US Market    0.061875
dtype: float64

In [21]:
# Posterior Combined Return Vector (same as above, but this one is written following the real BLM formula)
posterior_return = np.linalg.inv(
    np.linalg.inv(tau * cov) + (P.T @ np.linalg.inv(omega) @ P)
) @ (
    (np.linalg.inv(tau * cov) @ implied_returns_eq) + (P.T @ np.linalg.inv(omega) @ Q)
)

In [25]:
# Optimization
ef = EfficientFrontier(posterior_return, cov)
ef.add_objective(objective_functions.L2_reg)
ef.max_sharpe()
ef.clean_weights()

OrderedDict([('Global Bonds (Unhedged)', 0.0),
             ('Total US Bond Market', 0.0),
             ('US Large Cap Growth', 0.19455),
             ('US Large Cap Value', 0.11796),
             ('US Small Cap Growth', 0.15537),
             ('US Small Cap Value', 0.06887),
             ('Emerging Markets', 0.27181),
             ('Intl Developed ex-US Market', 0.19143)])

In [29]:
# Optimization
ef = EfficientFrontier(posterior_return, cov)
ef.add_objective(objective_functions.L2_reg)
ef.min_volatility()
ef.clean_weights()

OrderedDict([('Global Bonds (Unhedged)', 0.14031),
             ('Total US Bond Market', 0.14492),
             ('US Large Cap Growth', 0.11996),
             ('US Large Cap Value', 0.12265),
             ('US Small Cap Growth', 0.11934),
             ('US Small Cap Value', 0.12419),
             ('Emerging Markets', 0.11014),
             ('Intl Developed ex-US Market', 0.1185)])

<h1 style="color:orange">Pypfopt's Black-Litterman</h1>

In [36]:
# cov = risk_models.CovarianceShrinkage(excess_returns).ledoit_wolf()
# cov
excess_returns.isnull().sum()

Global Bonds (Unhedged)        0
Total US Bond Market           0
US Large Cap Growth            0
US Large Cap Value             0
US Small Cap Growth            0
US Small Cap Value             0
Emerging Markets               0
Intl Developed ex-US Market    0
dtype: int64

In [33]:
cov